# get(url)

Some great [Stata-to-Python](https://www.danielmsullivan.com/pages/tutorial_stata_to_python.html) code


Wednesday May 17,  12:00AM
```
(base) d@Poseidon 2.pantheon % streamlit run capstone.py

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.0.0.250:8501

```